# Importing SpaCy in English

In [1]:
import spacy
nlp = spacy.load("en_core_web_lg")


# trying it out on a small text

In [2]:
text = "My best friend Ryan Peters likes fancy adventure games."
doc = nlp(text)
for token in doc:
    print(token, end=" | ")


My | best | friend | Ryan | Peters | likes | fancy | adventure | games | . | 

# the attributes that SpaCy adds

In [3]:
import pandas as pd

def display_nlp(doc, include_punct=False):
    """Generate data frame for visualization of spaCy tokens."""
    rows = []
    for i, t in enumerate(doc):
        if not t.is_punct or include_punct:
            row = {'token': i,  'text': t.text, 'lemma_': t.lemma_, 
                   'is_stop': t.is_stop, 'is_alpha': t.is_alpha,
                   'pos_': t.pos_, 'dep_': t.dep_, 
                   'ent_type_': t.ent_type_, 'ent_iob_': t.ent_iob_}
            rows.append(row)
    
    df = pd.DataFrame(rows).set_index('token')
    df.index.name = None
    return df
display_nlp(doc)


,text,lemma_,is_stop,is_alpha,pos_,dep_,ent_type_,ent_iob_
0,My,my,True,True,PRON,poss,,O
1,best,good,False,True,ADJ,amod,,O
2,friend,friend,False,True,NOUN,nsubj,,O
3,Ryan,Ryan,False,True,PROPN,compound,PERSON,B
4,Peters,Peters,False,True,PROPN,appos,PERSON,I
5,likes,like,False,True,VERB,ROOT,,O
6,fancy,fancy,False,True,ADJ,amod,,O
7,adventure,adventure,False,True,NOUN,compound,,O
8,games,game,False,True,NOUN,dobj,,O


# Removing Stop words using Spacy

In [4]:
text = "Dear Ryan, we need to sit down and talk. Regards, Pete"
doc = nlp(text)

non_stop = [t for t in doc if not t.is_stop and not t.is_punct]
print(non_stop)


[Dear, Ryan, need, sit, talk, Regards, Pete]


# Find all nouns using Spacy

In [5]:
text = "My best friend Ryan Peters likes fancy adventure games."
doc = nlp(text)

nouns = [t for t in doc if t.pos_ in ['NOUN', 'PROPN']]
print(nouns)


[friend, Ryan, Peters, adventure, games]


# Named Entity Recognition

In [6]:
text = "My best friend Ryan Peters likes fancy adventure games."
doc = nlp(text)

for ent in doc.ents:
    print(f"({ent.text}, {ent.label_})", end=" ")


(Ryan Peters, PERSON) 

In [7]:
text = "James O'Neill, chairman of World Cargo Inc, lives in San Francisco." 
doc = nlp(text)

for ent in doc.ents:
    print(f"({ent.text}, {ent.label_})", end=" ")


(James O'Neill, PERSON) (World Cargo Inc, ORG) (San Francisco, GPE) 

# Visualize NERS

In [8]:
from spacy import displacy

displacy.render(doc, style='ent', jupyter=True)


# We will pull an article from the web and use it as our data, a reuters new article

In [9]:
from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))
ny_bb = url_to_string('https://tamil.news18.com/special-articles/')
article = nlp(ny_bb)
len(article.ents)


97

In [10]:
ny_bb

"News Stories of Featured Articles in Tamil, தமிழில் சிறப்புக் கட்டுரைகளின் செய்திகள்தமிழ்हिन्दीENGLISHবাংলাमराठीગુજરાતીঅসমীয়াಕನ್ನಡമലയാളംతెలుగుਪੰਜਾਬੀاردوଓଡ଼ିଆLIVE TVNEWS18 TAMILNEWS18 इंडिया LIVE TVNEWS18 LOKMATNews18 APP DOWNLOADதமிழ்நாடுசினிமாலைஃப்ஸ்டைல்வணிகம்விளையாட்டுஇந்தியாஉலகம்Live TVதமிழ்நாடுசினிமாலைஃப்ஸ்டைல்வணிகம்விளையாட்டுஇந்தியாஉலகம்Live TVLatest News உள்ளூர்ஐபிஎல்2024தொழில்நுட்பம்லோக்சபா 2024போட்டோட்ரெண்ட்வேலைகல்வி ஆன்மிகம்Rising Bharatஉங்கள் நகரத்தைத் தேர்ந்தெடுக்கவும்அரியலூர்செங்கல்பட்டுசென்னைகோயம்புத்தூர்கடலூர்தர்மபுரிதிண்டுக்கல்ஈரோடுகள்ளக்குறிச்சிகாஞ்சிபுரம்கன்னியாகுமரிகரூர்கிருஷ்ணகிரிமதுரைமயிலாடுதுறைநாகப்பட்டினம்நாமக்கல்நீலகிரிபெரம்பலூர்புதுச்சேரிபுதுக்கோட்டைராமநாதபுரம்ராணிப்பேட்டைசேலம்சிவகங்கைதென்காசிதஞ்சாவூர்தேனிதூத்துக்குடிதிருநெல்வேலிதிருப்பத்தூர்திருப்பூர்திருவள்ளூர்திருவண்ணாமலைதிருவாரூர்திருச்சிவேலூர்விழுப்புரம்விருதுநகர்Latest News உள்ளூர்ஐபிஎல்2024தொழில்நுட்பம்லோக்சபா 2024போட்டோட்ரெண்ட்வேலைகல்வி ஆன்மிகம்Rising BharatCHOOSE YOUR DISTRICTவிளம்பரம்தமிழ் செய்திகள் 

# the NERS

In [11]:
displacy.render(article, style='ent', jupyter=True)


# Most popular NER type

In [12]:
from collections import Counter

# Assuming `article` is your SpaCy document
labels = [x.label_ for x in article.ents]
counter = Counter(labels)
print(counter)


Counter({'ORG': 33, 'PERSON': 19, 'CARDINAL': 16, 'GPE': 12, 'DATE': 5, 'NORP': 4, 'LAW': 2, 'ORDINAL': 2, 'FAC': 1, 'PRODUCT': 1, 'EVENT': 1, 'LANGUAGE': 1})


In [13]:
items = [x.text for x in article.ents]
Counter(items).most_common(5)


[('4', 3), ('100', 3), ('ராசியினருக்கு', 3), ('Krishnasamy', 3), ('CNN', 3)]

# Most popular NER

In [14]:
items = [x.text for x in article.ents]
Counter(items).most_common(5)


[('4', 3), ('100', 3), ('ராசியினருக்கு', 3), ('Krishnasamy', 3), ('CNN', 3)]

# Taking One sentence to analyze

In [15]:
sentences = [x for x in article.sents]
print(sentences[0])


News Stories of Featured Articles in Tamil, தமிழில் சிறப்புக் கட்டுரைகளின் செய்திகள்தமிழ்हिन्दीENGLISHবাংলাमराठीગુજરાતીঅসমীয়াಕನ್ನಡമലയാളംతెలుగుਪੰਜਾਬੀاردوଓଡ଼ିଆLIVE TVNEWS18 TAMILNEWS18


# NER tags

In [16]:
displacy.render(nlp(str(sentences[0])), jupyter=True, style='ent')


# Types of words in the sentence

In [17]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(str(sentences[0])) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]


[('News', 'NOUN', 'news'),
 ('Stories', 'PROPN', 'Stories'),
 ('Featured', 'VERB', 'feature'),
 ('Articles', 'PROPN', 'Articles'),
 ('Tamil', 'PROPN', 'Tamil'),
 ('தமிழில்', 'X', 'தமிழில்'),
 ('சிறப்புக்', 'X', 'சிறப்புக்'),
 ('கட்டுரைகளின்', 'VERB', 'கட்டுரைகளின்'),
 ('செய்திகள்தமிழ்हिन्दीENGLISHবাংলাमराठीગુજરાતીঅসমীয়াಕನ್ನಡമലയാളംతెలుగుਪੰਜਾਬੀاردوଓଡ଼ିଆLIVE',
  'ADJ',
  'செய்திகள்தமிழ்हिन्दीenglishবাংলাमराठीગુજરાતીঅসমীয়াಕನ್ನಡമലയാളംతెలుగుਪੰਜਾਬੀاردوଓଡ଼ିଆlive'),
 ('TVNEWS18', 'NOUN', 'tvnews18')]

# Sentence dependency tree

In [18]:
displacy.render(nlp(str(sentences[0])), style='dep', jupyter = True, options = {'distance': 120})
